In [19]:
import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

C:\Users\yxu94\AppData\Local\Programs\Python\Python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\yxu94\AppData\Local\Programs\Python\Python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
C:\Users\yxu94\AppData\Local\Programs\Python\Python37\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [25]:
import pandas as pd
import datetime

In [4]:
logdir = 'logs/hparam_tuning'
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
  hp.hparams_config(
    hparams=[HP_OPTIMIZER],
    metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
  )

In [44]:
# 一些超参
to_date = datetime.datetime.now().strftime("%Y-%m-%d")      # 今日日期       
re_download = True              # 重新下载数据
re_train = True                 # 重新训练
predict_period = 15             # 预测天数
history_period = 400            # 分析天数
epoch = 200
start_date = '2010-01-01'       # 最早数据
threshold_flat = 0.007           # 股价持平的阈值
stock_code_list = pd.read_csv('stock_codes.csv')['code']    # 获取需要预测的股票代码

# 一些设置
verify_period = 2              # 验证周期

i = 0
stock_code = 'sz.000858'
stock_info_path = "stock_info/" + stock_code + ".csv"       # 文件路径
# 读取csv文件
stock = pd.read_csv(stock_info_path, parse_dates=['date'])

if i == 0:
    pass
else:
    stock = stock[:-i]

# 准备数据
stock['close_nomalized'] = (stock['close']-stock['close'].min())/(stock['close'].max()-stock['close'].min())        # 收盘价 归一化
stock['future_price'] = stock['close'].rolling(predict_period).mean().shift(-predict_period)                        # 未来股价均值(不包含当日收盘价)

def flat_or_not(x):
    if x >= threshold_flat:
        return 2       # 涨
    elif x <= -threshold_flat:
        return 1       # 跌
    else:
        return 0       # 持平

stock['label'] = ((stock['future_price'] - stock['close']) / stock['close']).apply(flat_or_not)

n = len(stock)
x = np.array([stock[['close_nomalized', 'turn']][i:i+history_period] for i in range(n-history_period-predict_period+1)]).reshape(-1, 20, 20, 2) # 输入 400天
x = x[:, :, :, :, np.newaxis]
y = stock['label'][history_period-1:].values[:-predict_period]                                               # 标签 

print(pd.DataFrame(y)[0].value_counts())    # 打印三种类别样本的个数。


2    1160
1     874
0     297
Name: 0, dtype: int64


In [39]:
np.array([stock[['close_nomalized', 'turn']][i:i+history_period] for i in range(n-history_period-predict_period+1)]).shape

(2331, 400, 2)

In [46]:
x

array([[[[[0.05146546],
          [0.695795  ]],

         [[0.05246567],
          [0.766646  ]],

         [[0.04991968],
          [1.128226  ]],

         ...,

         [[0.04270603],
          [0.747497  ]],

         [[0.04346377],
          [0.614937  ]],

         [[0.04461552],
          [0.478751  ]]],


        [[[0.04631285],
          [0.763156  ]],

         [[0.0438881 ],
          [0.552596  ]],

         [[0.04428212],
          [0.601669  ]],

         ...,

         [[0.04225139],
          [0.499899  ]],

         [[0.04161489],
          [0.745284  ]],

         [[0.04188767],
          [0.381774  ]]],


        [[[0.04161489],
          [0.310684  ]],

         [[0.04112994],
          [0.52691   ]],

         [[0.03952354],
          [0.638421  ]],

         ...,

         [[0.04152396],
          [0.506467  ]],

         [[0.04255448],
          [0.505175  ]],

         [[0.04191798],
          [0.482984  ]]],


        ...,


        [[[0.05883066],
          

In [2]:
def train_test_model(hparams):
    model = keras.models.Sequential()
    model.add(keras.layers.Conv2D(32, 3, activation='relu', input_shape=(20, 20, 1)))         # 卷积核的个数 => 输出的维度
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))
    model.add(keras.layers.MaxPooling2D((2, 2)))
    model.add(keras.layers.Conv2D(64, (3, 3), activation='relu'))

    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dense(3))
    model.compile(optimizer='adam', 
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=['accuracy'])
            
    model.fit(x_train, y_train, epochs=epoch, validation_data=(x_test, y_test), initial_epoch=10, callbacks = [tf.keras.callbacks.TensorBoard(logdir), hp.KerasCallback(logdir, hparams)])
    _, accuracy = model.evaluate(x_test, y_test)
    return accuracy

In [ ]:
%tensorboard --logdir logs/hparam_tuning